In [1]:
import random

In [6]:
class HumanBot:
    def __init__(self, name, chips):
      self.name = name
      self.card = None  # Initially, the card is not set
      self.chips = chips
      self.bet = 0

    def set_bet(self, amount):
      self.chips = self.chips - amount
      self.bet = self.bet + amount

    def display_info(self):
      return f"{self.name}: Card - {self.card}, Chips - {self.chips}, Bet - {self.bet}"

    def display_chips(self):
      return f"{self.name}: Chips - {self.chips}"

    def get_decision(self, state):
      user_input = input(f"Your card is {self.card}. Would you like to bet? (y or n)")
      if user_input == "y":
        return 1
      else:
        return 0


In [3]:
class PokerBot:
    def __init__(self, name, chips):
      self.name = name
      self.card = None  # Initially, the card is not set
      self.chips = chips
      self.bet = 0

      state_1_strategies = {}
      state_1_strategies['K'] = 0.12
      state_1_strategies['Q'] = 0.02
      state_1_strategies['J'] = 0.05

      state_2_strategies = {}
      state_2_strategies['K'] = 1
      state_2_strategies['Q'] = 0.22
      state_2_strategies['J'] = 0

      state_3_strategies = {}
      state_3_strategies['K'] = 1
      state_3_strategies['Q'] = 0
      state_3_strategies['J'] = 0.29

      state_4_strategies = {}
      state_4_strategies['K'] = 1
      state_4_strategies['Q'] = 0.07
      state_4_strategies['J'] = 0

      self.strategies = {}
      self.strategies[1] = state_1_strategies
      self.strategies[2] = state_2_strategies
      self.strategies[3] = state_3_strategies
      self.strategies[4] = state_4_strategies

    def set_bet(self, amount):
      self.chips = self.chips - amount
      self.bet = self.bet + amount

    def display_info(self):
      return f"{self.name}: Card - {self.card}, Chips - {self.chips}, Bet - {self.bet}"

    def display_chips(self):
      return f"{self.name}: Chips - {self.chips}"

    def get_decision(self, state):
      bet_chance = self.strategies[state][self.card]
      if random.random() < bet_chance:
        return 1
      else:
        return 0


In [4]:
class PokerGame:
    def __init__(self, bot1, bot2, verbose):
      self.dealer = bot1
      self.blind = bot2
      self.deck = ['K','Q','J']
      self.winner = 0
      self.round = 1
      self.verbose = verbose
      self.game_data = {}
      self.current_round = ""

    def reset_game(self, amount):
      self.dealer.chips = amount
      self.blind.chips = amount
      self.dealer.bet = 0
      self.blind.bet = 0
      self.dealer.card = None
      self.blind.card = None
      self.deck = ['K','Q','J']
      self.current_round = ""

    def announce(self, message):
      if self.verbose == 1:
        print(message)

    def update_positions(self, bot1, bot2):
      self.dealer = bot1
      self.blind = bot2
      self.deck = ['K','Q','J']
      self.winner = 0

    def deal_cards(self):
      self.deck = ['K','Q','J']
      random.shuffle(self.deck)
      self.dealer.card = self.deck.pop()
      self.blind.card = self.deck.pop()
      self.current_round = self.dealer.card + self.blind.card

    def pay_blinds(self):
      self.dealer.set_bet(5)
      self.blind.set_bet(5)

    def play_round(self):
      if self.dealer.chips > 0 and self.blind.chips > 0:
        self.announce(f"round {self.round}")
        self.display_score()
        self.deal_cards()
        self.pay_blinds()
        self.state1()
        self.showdown()
        self.update_chips()
        self.update_positions(self.blind, self.dealer)
        self.round = self.round + 1
        return 0
      elif self.blind.chips <= 0:
        self.announce(f"{self.dealer.name} won")
        return 1
      elif self.dealer.chips <= 0:
        self.announce(f"{self.blind.name} won")
        return 2

    def state1(self):
      decision = self.dealer.get_decision(1)
      if decision == 1:
        self.announce(f"{self.dealer.name} bet")
        self.dealer.set_bet(10)
        self.current_round = self.current_round + "B"
        self.state2()
      else:
        self.announce(f"{self.dealer.name} checked")
        self.current_round = self.current_round + "F"
        self.state3()

    def state2(self):
      decision = self.blind.get_decision(2)
      if decision == 1:
        self.announce(f"{self.blind.name} bet")
        self.blind.set_bet(10)
        self.current_round = self.current_round + "B"
        self.winner = self.determine_winner()
      else:
        self.announce(f"{self.blind.name} folded")
        self.current_round = self.current_round + "F"
        self.winner = 1

    def state3(self):
      decision = self.blind.get_decision(3)
      if decision == 1:
        self.announce(f"{self.blind.name} bet")
        self.blind.set_bet(10)
        self.current_round = self.current_round + "B"
        self.state4()
      else:
        self.announce(f"{self.blind.name} checked")
        self.current_round = self.current_round + "F"
        self.winner = self.determine_winner()

    def state4(self):
      decision = self.dealer.get_decision(4)
      if decision == 1:
        self.announce(f"{self.dealer.name} bet")
        self.dealer.set_bet(10)
        self.current_round = self.current_round + "B"
        self.winner = self.determine_winner()
      else:
        self.announce(f"{self.dealer.name} folded")
        self.current_round = self.current_round + "F"
        self.winner = 2

    def showdown(self):
      self.announce(self.dealer.display_info())
      self.announce(self.blind.display_info())

    def display_score(self):
      self.announce(self.dealer.display_chips())
      self.announce(self.blind.display_chips())

    def update_chips(self):
      pot = self.dealer.bet + self.blind.bet

      if self.winner == 1:
        self.current_round = self.current_round + f" Won {pot - self.dealer.bet}"
        self.announce(f"{self.dealer.name} won {pot - self.dealer.bet} chips")
        self.dealer.chips = self.dealer.chips + pot
      elif self.winner == 2:
        self.current_round = self.current_round + f" Lost {pot - self.blind.bet}"
        self.announce(f"{self.blind.name} won {pot - self.blind.bet} chips")
        self.blind.chips = self.blind.chips + pot
      self.dealer.bet = 0
      self.blind.bet = 0
      if self.current_round in self.game_data:
        self.game_data[self.current_round] += 1
      else:
        self.game_data[self.current_round] = 1

    def determine_winner(self):
      player1_card, player2_card = self.dealer.card, self.blind.card
      card_ranking = {'K':3, 'Q':2, 'J':1}
      if card_ranking[player1_card] > card_ranking[player2_card]:
        return 1
      else:
        return 2

In [9]:
player1 = PokerBot("pokerbot", 100)
player2 = HumanBot("humanbot", 100)
game = PokerGame(player1, player2, 1)

game.reset_game(100)
while True:
  result = game.play_round()

  if result != 0:
    break

round 1
pokerbot: Chips - 100
humanbot: Chips - 100
pokerbot checked
Your card is K. Would you like to bet? (y or n)y
humanbot bet
pokerbot folded
pokerbot: Card - J, Chips - 95, Bet - 5
humanbot: Card - K, Chips - 85, Bet - 15
humanbot won 5 chips
round 2
humanbot: Chips - 105
pokerbot: Chips - 95


KeyboardInterrupt: Interrupted by user